In [ ]:
import socket
import threading
from tkinter import Tk, Button, Label
from tkinter import messagebox

# Server configuration
SERVER_IP = '0.0.0.0'  # Bind to all interfaces
SERVER_PORT = 12348
BUFFER_SIZE = 1024

# TCP Server and connection thread
server_socket = None
client_socket = None
server_thread = None
is_server_running = False

# LED states
led_states = {
    'led1': False,
    'led2': False,
    'led3': False,
    'led4': False,
    'led5': False
}

def start_server():
    global server_socket, client_socket, is_server_running
    try:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((SERVER_IP, SERVER_PORT))
        server_socket.listen(1)
        is_server_running = True
        print("Server started, waiting for connections...")

        while is_server_running:
            client_socket, _ = server_socket.accept()
            print("Client connected")
            
            while is_server_running:
                try:
                    data = client_socket.recv(BUFFER_SIZE).decode().strip()
                    if data:
                        print(f"Received data: {data}")
                        if data == "GET_INPUTS":
                            response = f"LED1_{'ON' if led_states['led1'] else 'OFF'}"
                            client_socket.sendall((response + '\n').encode())
                except socket.error as e:
                    print(f"Socket error: {e}")
                    break

            client_socket.close()
    except socket.error as e:
        print(f"Socket error: {e}")
    finally:
        if server_socket:
            server_socket.close()

def stop_server():
    global is_server_running
    is_server_running = False
    if client_socket:
        client_socket.close()
    if server_socket:
        server_socket.close()
    print("Server stopped")

def control_led(led_id, action):
    global client_socket
    led_states[led_id] = True if action == 'on' else False

    if client_socket and is_valid_socket(client_socket):
        command = f"{led_id.upper()}_{action.upper()}"
        try:
            client_socket.sendall((command + '\n').encode())
        except socket.error as e:
            print(f"Error sending command: {e}")
            messagebox.showerror("Error", f"Error sending command: {e}")
    else:
        messagebox.showerror("Error", "No valid client connection")

def is_valid_socket(sock):
    try:
        sock.send(b"")
    except socket.error:
        return False
    return True

def start_server_ui():
    if not is_server_running:
        global server_thread
        server_thread = threading.Thread(target=start_server)
        server_thread.start()
        messagebox.showinfo("Info", "Server started")
    else:
        messagebox.showinfo("Info", "Server already running")

def stop_server_ui():
    if is_server_running:
        stop_server()
        messagebox.showinfo("Info", "Server stopped")
    else:
        messagebox.showinfo("Info", "Server not running")

def create_ui():
    root = Tk()
    root.title("ESP32 LED Control")

    Label(root, text="Server Controls").grid(row=0, columnspan=2)
    Button(root, text="Start Server", command=start_server_ui).grid(row=1, column=0)
    Button(root, text="Stop Server", command=stop_server_ui).grid(row=1, column=1)

    Label(root, text="LED Controls").grid(row=2, columnspan=2)

    led_buttons = [
        ("LED 1 ON", "led1", "on"), ("LED 1 OFF", "led1", "off"),
        ("LED 2 ON", "led2", "on"), ("LED 2 OFF", "led2", "off"),
        ("LED 3 ON", "led3", "on"), ("LED 3 OFF", "led3", "off"),
        ("LED 4 ON", "led4", "on"), ("LED 4 OFF", "led4", "off"),
        ("LED 5 ON", "led5", "on"), ("LED 5 OFF", "led5", "off")
    ]

    for i, (text, led_id, action) in enumerate(led_buttons):
        Button(root, text=text, command=lambda l=led_id, a=action: control_led(l, a)).grid(row=3 + i // 2, column=i % 2)

    root.mainloop()

if __name__ == '__main__':
    create_ui()
